# Global Food Prices

Get dataset

In [ ]:
import os
from download import download
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

url_2 = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
regional_file_name = "regions.csv"

url = "http://vam.wfp.org/sites/data/WFPVAM_FoodPrices_05-12-2017.csv"
file_name = 'WFPVAM_FoodPrices_05-12-2017.csv'

def get_dataset(url, file_name):
    if not os.path.isfile(file_name):
        download(url, file_name)
    
    return pd.read_csv(file_name, encoding='latin-1')

df = get_dataset(url, file_name)

region_df = get_dataset(url_2, regional_file_name)

In [ ]:
# Pre-process
region_df.rename(columns={'name': 'adm0_name'}, inplace=True)
new_regions = region_df.loc[:, ['adm0_name', 'sub-region']]

df_regions = pd.merge(df, new_regions, on='adm0_name', how='left')

df = df_regions.copy()

df['datetime'] = pd.to_datetime(df.mp_year*10000+df.mp_month*100+1, format='%Y%m%d')


In [ ]:
import numpy as np
avg_price = df.groupby(['adm0_name', 'cm_name', 'datetime'])['mp_price']
df_means = avg_price.mean().reset_index()


normalized = df_means.groupby(['adm0_name', 'cm_name'])['mp_price'].apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

In [ ]:
df_means['mp_price_norm'] = normalized

# only_apples = df_means.loc[df_means['sub-region'] == 'Central Asia'].loc[df_means['cm_name'] == 'Apples']
# print("MAX", only_apples['mp_price'].max(), "MIN", only_apples['mp_price'].min(), "MEAN", only_apples['mp_price'].mean())

In [ ]:
# plt.plot(df_means['datetime'], df_means['mp_price'], '-')

# Make a graph per product. Each line represents a country (or region for later).


from bokeh.io import output_file, show, save
# output_notebook()

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure

In [ ]:
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.layouts import column

b = df_means.groupby(['cm_name'])

figures = []

idx = 0
for group, row in b:
    color_idx = 0
    country_group = row.groupby(['adm0_name'])
    p = figure(plot_width=800, plot_height=300, x_axis_type='datetime')
    p.title.text = group
    p.legend.click_policy="hide"
    p.legend.location = "top_left"
    
    for group2, row2 in country_group:
        datetime = []
        prices = []
        for i, data in row2.iterrows():
            datetime.append(data['datetime'])
            prices.append(data['mp_price_norm'])
        p.line(datetime, prices, line_width=4, legend=data['adm0_name'], color=palette[color_idx % 5])
        color_idx += 1
    figures.append(p)
    idx += 1
    if idx > 20:
        break

output_file("groups" + ".html")
show(column(figures))


In [102]:
df

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,sub-region,datetime
0,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,1.0,2014.0,50.00,WFP,Southern Asia,2014-01-01
1,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,2.0,2014.0,50.00,WFP,Southern Asia,2014-02-01
2,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,3.0,2014.0,50.00,WFP,Southern Asia,2014-03-01
3,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,4.0,2014.0,50.00,WFP,Southern Asia,2014-04-01
4,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,5.0,2014.0,50.00,WFP,Southern Asia,2014-05-01
5,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,6.0,2014.0,50.00,WFP,Southern Asia,2014-06-01
6,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,7.0,2014.0,50.00,WFP,Southern Asia,2014-07-01
7,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,8.0,2014.0,50.00,WFP,Southern Asia,2014-08-01
8,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,9.0,2014.0,50.00,WFP,Southern Asia,2014-09-01
9,1,Afghanistan,272,Badakhshan,266.0,Fayzabad,55.0,Bread,87.0,AFN,15.0,Retail,5.0,KG,10.0,2014.0,50.00,WFP,Southern Asia,2014-10-01


In [ ]:
df.groupby('cm_name').cm_name.count()



In [ ]:
df.cm_name.unique()

In [ ]:
avg_price = df.groupby(['adm0_name', 'cm_name'])['mp_price']
df_means = avg_price.mean().reset_index()

df_means

In [104]:
for product in df.cm_name.unique():
    print(product)
    dates = df.loc[df.cm_name == product,'datetime']
    minimum = min(dates)
    maximum = max(dates)
    print('Min:') 
    print(minimum)
    print('Max:')
    print(maximum)
    country = df.loc[df.cm_name == product,'adm0_name'].unique()
    #df.cm_name.unique()
    print(country)

    
#    country = df.loc[df.cm_name == product,'adm0_name']
   
#    country = df.ix[(df.cm_name == product).idxmin(), 'adm0_name']


#minimum en maximum voor apples
#landen weten die apples hebben geregistreerd
# if datum van land > min of datum < maximum 
# vul aan tot minimum of maximum met het gemiddelde (cellen toevoegen)



Bread
Min:
2001-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Afghanistan' 'Algeria' 'Congo' 'Djibouti' 'Gambia' 'Guatemala' 'Guinea'
 'Kenya' 'Kyrgyzstan' 'Nigeria']
Wheat
Min:
1994-01-01 00:00:00
Max:
2017-11-01 00:00:00
['Afghanistan' 'Ethiopia' 'Guinea-Bissau' 'India' 'Indonesia' 'Kyrgyzstan'
 'Mauritania' 'Nepal' 'Nigeria' 'Pakistan' 'Rwanda']
Rice (low quality)
Min:
2007-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Afghanistan' 'Myanmar']
Wage (qualified labour)
Min:
2013-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Afghanistan' 'Kyrgyzstan']
Livestock (sheep, one-year-old alive female)
Min:
2000-05-01 00:00:00
Max:
2002-12-01 00:00:00
['Afghanistan']
Fuel (diesel)
Min:
1996-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Afghanistan' 'Algeria' 'Armenia' 'Colombia'
 'Democratic Republic of the Congo' 'Ethiopia' 'Gambia' 'Guatemala'
 'Kenya' 'Kyrgyzstan' "Lao People's Democratic Republic" 'Liberia'
 'Madagascar' 'Myanmar' 'Nigeria' 'Pakistan' 'Somalia' 'Sri Lanka'
 'Syrian Arab Republic']
Exchang

Min:
1992-01-01 00:00:00
Max:
2017-11-01 00:00:00
['Benin' 'Burkina Faso' 'Central African Republic' 'Chad' 'Djibouti'
 'Gambia' 'Guinea-Bissau' 'Mali' 'Niger' 'Nigeria' 'Senegal']
Rice (local)
Min:
2003-10-01 00:00:00
Max:
2017-11-01 00:00:00
['Bhutan' 'Cameroon' 'Chad' "Cote d'Ivoire"
 'Democratic Republic of the Congo' 'Ghana' 'Guinea-Bissau' 'Guinea'
 'Haiti' 'Iran  (Islamic Republic of)' 'Madagascar' 'Mali' 'Mauritania'
 'Mozambique' 'Niger' 'Nigeria' 'Peru' 'Rwanda' 'Senegal']
Rice (imported, Indian)
Min:
2006-03-01 00:00:00
Max:
2015-12-01 00:00:00
['Bhutan' 'Rwanda']
Meat (beef, chops with bones)
Min:
2008-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Bolivia' 'Guatemala' 'Philippines']
Meat (chicken, whole)
Min:
2008-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Bolivia' 'Jordan' 'Lebanon' 'Philippines']
Noodles (short)
Min:
2010-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Bolivia']
Potatoes (Dutch)
Min:
2010-01-01 00:00:00
Max:
2012-05-01 00:00:00
['Bolivia']
Potatoes (Irish, imilla)
M

Min:
2001-01-01 00:00:00
Max:
2017-09-01 00:00:00
['Costa Rica' 'Guatemala' 'Haiti']
Rice (milled 80-20)
Min:
2007-09-01 00:00:00
Max:
2017-05-01 00:00:00
['Costa Rica' 'Honduras' 'Panama']
Rice (denikassia, imported)
Min:
2005-03-01 00:00:00
Max:
2017-07-01 00:00:00
["Cote d'Ivoire"]
Yam (florido)
Min:
2006-01-01 00:00:00
Max:
2017-07-01 00:00:00
["Cote d'Ivoire"]
Cornstarch
Min:
2011-01-01 00:00:00
Max:
2017-07-01 00:00:00
["Cote d'Ivoire"]
Fish (appolo)
Min:
2011-01-01 00:00:00
Max:
2017-07-01 00:00:00
["Cote d'Ivoire"]
Peanut
Min:
2011-01-01 00:00:00
Max:
2017-07-01 00:00:00
["Cote d'Ivoire"]
Groundnuts
Min:
2016-12-01 00:00:00
Max:
2017-02-01 00:00:00
["Cote d'Ivoire"]
Cashew
Min:
2013-05-01 00:00:00
Max:
2017-09-01 00:00:00
["Cote d'Ivoire" 'Gambia']
Cotton
Min:
2017-03-01 00:00:00
Max:
2017-03-01 00:00:00
["Cote d'Ivoire"]
Maize meal
Min:
2008-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Democratic Republic of the Congo' 'Lesotho' 'Swaziland']
Cassava
Min:
2008-01-01 00:00:00
Max:


Min:
2014-01-01 00:00:00
Max:
2017-10-01 00:00:00
["Lao People's Democratic Republic"]
Rice (imported, Egyptian)
Min:
2012-08-01 00:00:00
Max:
2016-12-01 00:00:00
['Lebanon']
Lettuce
Min:
2013-09-01 00:00:00
Max:
2016-12-01 00:00:00
['Lebanon']
Meat (beef, canned)
Min:
2012-08-01 00:00:00
Max:
2016-12-01 00:00:00
['Lebanon']
Fish (sardine, canned)
Min:
2013-09-01 00:00:00
Max:
2017-09-01 00:00:00
['Lebanon' 'Syrian Arab Republic']
Bread (brown)
Min:
2010-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Lesotho']
Peas (split, dry)
Min:
2010-08-01 00:00:00
Max:
2017-10-01 00:00:00
['Lesotho']
Beans (sugar-red)
Min:
2010-01-01 00:00:00
Max:
2017-10-01 00:00:00
['Lesotho']
Cowpeas
Min:
2011-05-01 00:00:00
Max:
2017-10-01 00:00:00
['Liberia' 'Mozambique']
Rice (white, imported)
Min:
2009-01-01 00:00:00
Max:
2017-09-01 00:00:00
['Liberia']
Wage (non-qualified labour, agricultural)
Min:
2014-09-01 00:00:00
Max:
2017-06-01 00:00:00
['Liberia']
Sorghum (taghalit)
Min:
2008-01-01 00:00:00
Max:
2017-10-

ValueError: min() arg is an empty sequence